In [ ]:
#import TtST_data_test.csv as teh test dataframe
import pandas as pd
from openai import 
AI
import openai

# Load the data
df_llm = pd.read_csv('TtST_llm_tabular_text.csv')

openai.api_key ='open ai key'

client = OpenAI(api_key = openai.api_key)


In [9]:
#performa  train test split
from sklearn.model_selection import train_test_split
# Prepare features and target variable
target = df_llm['Issue Lifetime']  # This will be treated as a classification target
features = df_llm.drop(['Issue Lifetime', 'Issue_Lifetime(hours)','pr_number'], axis=1, errors='ignore')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#combine train and test sets
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)



## Helper Method to generate prompts

In [10]:
#print out al teh columns
print("Training Data Columns:")
for col in df_train.columns:
    print(col)

Training Data Columns:
number_of_comments
number_of_review_comments
number_of_commits
lines_of_code_changed
number_of_files_changed
number_of_reviewers
number_of_approvals
labels
time_to_first_response
number_of_assignees
review_duration
number_of_changes_requested
number_of_build_runs
number_of_build_failures
number_of_reviews_requested
number_of_revisions
issue_comments_text
issue_text
review_comments
repo
comments
comment_author_number
issue_text_wordiness
issue_comments_text_length
issue_comments_text_wordiness
code_changes_ratio
label_count
day_of_week
weekday
Author PRs Opened
Open PRs at Open Date
Issues Closed in Last 2 Weeks
Issues Opened in Last 2 Weeks
Issue Lifetime


In [11]:
# Generate the feature string dynamically
excluded_columns = ['Issue Lifetime', 'number_of_reviewers', 'number_of_approvals', 'number_of_changes_requested', 'number_of_build_runs','number_of_build_failures','review_comments','number_of_revisions']
def generate_prompt(row):
    feature_str = "\n".join([f"- {col}: {row[col]}" for col in row.index if col not in excluded_columns])
    
    prompt = (
        "Classify, using ONLY 1 NUMBER between 1 and 5, the following GitHub issue, where:\n"
        "1 = 'Less than 1 hour'\n"
        "2 = 'Between 1 hour and 1 day'\n"
        "3 = 'Between 1 day and a week'\n"
        "4 = 'Between 1 week and 1 month'\n"
        "5 = 'More than a month'\n"
        f"Based on its features:\n{feature_str}"
    )
    
    return prompt

## Creating training file

In [10]:
import json

# Open the file in write mode
with open('LLM_tabular_training_data.jsonl', 'w', encoding='utf-8') as f:
    # Iterate over the rows in the DataFrame
    for index, row in df_train.iterrows():
        # Create the user message by formatting the prompt with the values from each column for each row using the generate_prompt function
        user_message = generate_prompt(row)
        
        
        
        # Create the assistant message by taking the label
        assistant_message = str(row['Issue Lifetime'])
        
        # Construct the conversation object
        conversation_object = {
            "messages": [
                {"role": "system", "content": "GitHub Issue Lifetime Classifier"},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }
        
        # Write the conversation object to one line in the file
        f.write(json.dumps(conversation_object, ensure_ascii=False) + '\n')

## Uplopading a training file


In [ ]:
"""ft_file = client.files.create(
  file=open("LLM_tabular_training_data.jsonl", "rb"),
  purpose="fine-tune"
)
ft_file""""

FileObject(id='file-EtPpp1UPkCtmGZQowCyNNz', bytes=24297974, created_at=1741286833, filename='LLM_tabular_training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

## Creating a finetuning job

In [ ]:
'''ft_job = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-4o-mini-2024-07-18",
  suffix= "issueLifetime"
)
ft_job

fine_tuning_job_id= ft_job.id'''


ftjob-NMEf0iQDY6DveP8HhhZITwvE


In [17]:
job_status = client.fine_tuning.jobs.retrieve('ftjob-NMEf0iQDY6DveP8HhhZITwvE')

if job_status.status == 'succeeded':
    fine_tuned_model = job_status.fine_tuned_model
    print("Fine-tuned model ID:", fine_tuned_model)
elif job_status.status == 'failed':
    print("Fine-tuning job has failed.")
else:
    print("Fine-tuning job has not completed yet.") 

Fine-tuned model ID: ft:gpt-4o-mini-2024-07-18:fabiocsu:issuelifetime:B8CFA3lh


## Using the Finetuned Model

In [18]:
fine_tuning_job_id = 'ftjob-NMEf0iQDY6DveP8HhhZITwvE'
tts_ft_model = client.fine_tuning.jobs.retrieve(fine_tuning_job_id).fine_tuned_model

In [19]:
import concurrent.futures
import tiktoken
import time


# max_token here should be one since the class (1,2,3,4) is one token long. 
def query_chatgpt(prompt, model, temperature=0.0,  max_tokens=1, max_retries=5):
    """
    Function to query ChatGPT-4 with a given prompt, with retries for timeouts.

    :param prompt: Prompt string to send to ChatGPT-2.5
    :param model: The model to use, default is ChatGPT-3.5
    :param max_tokens: Maximum number of tokens to generate
    :param max_retries: Maximum number of retries for timeout
    :return: Response from ChatGPT-4o-mini or None if all retries fail
    """
    attempt = 0
    max_content_tokens = 3999
    encoding = tiktoken.get_encoding("cl100k_base")
    encoding = tiktoken.encoding_for_model("gpt-4o-mini-2024-07-18")

    # Function to truncate the message and avoid passing the limit of 4k tokens per gpt-3.5 fine-tuned model limitations
    def truncate_message(message, max_length):
        tokens = encoding.encode(message)
        if len(tokens) > max_length:
            truncated_tokens = tokens[:max_length]
            message = encoding.decode(truncated_tokens)
        return message

    # Truncate the prompt if necessary
    prompt = truncate_message(prompt, max_content_tokens)

    while attempt < max_retries:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(
                openai.chat.completions.create,
                model=model,
                messages=[{"role": "system", "content": "GitHub Issue Lifetime Classifier"}, {"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=temperature
            )
            try:
                response = future.result(timeout=5)  # 5 seconds timeout
                return response.choices[0].message.content
            except concurrent.futures.TimeoutError:
                print(f"Attempt {attempt + 1}/{max_retries} - Request timed out. Retrying...")
            except Exception as e:
                print(f"Attempt {attempt + 1}/{max_retries} - An error occurred: {e}")
            finally:
                attempt += 1

    print("Failed to get a response after several retries.")
    return None
    
labels =[ 1, 2, 3, 4, 5]

In [20]:
y_true = []
y_pred = []

y_llm = []

iterations = len(df_test)
#lifetime = df_test['Issue Lifetime']
#predictors = df_test.drop('Issue Lifetime', axis=1)



# Now let's loop through the test data and classify the GitHub issues
for i, row in df_test.iterrows():
    correct_label = int(row['Issue Lifetime'])  # Access the correct label directly from the row
    description = row.drop('Issue Lifetime').to_string()  # Get the row data as a string

    print(f"Correct PR type: {correct_label}")
    
    # Generate the prompt
    prompt = generate_prompt(row)
    
    response = query_chatgpt(prompt, tts_ft_model)
    
    if response is None:
        print("Failed to get a response after several retries. Skipping this item.")
        continue  # Skip this iteration and move to the next one
    
    # Clean the response to keep only letters (and optionally numbers)
    predicted_label = int(response)
    print(f"Predicted PR Lifetime: {predicted_label}")
    
    if predicted_label == correct_label:
        y_llm.append(1)
    else:
        y_llm.append(0)
        
    # Append to lists for evaluation
    y_true.append(correct_label)
    y_pred.append(predicted_label)
    time.sleep(3)  # Wait for 3 seconds before retrying

Correct PR type: 1
Attempt 1/5 - Request timed out. Retrying...
Attempt 2/5 - Request timed out. Retrying...
Attempt 3/5 - Request timed out. Retrying...
Attempt 4/5 - Request timed out. Retrying...
Predicted PR Lifetime: 1
Correct PR type: 2
Predicted PR Lifetime: 2
Correct PR type: 3
Attempt 1/5 - Request timed out. Retrying...
Attempt 2/5 - Request timed out. Retrying...
Attempt 3/5 - Request timed out. Retrying...
Predicted PR Lifetime: 2
Correct PR type: 3
Predicted PR Lifetime: 3
Correct PR type: 4
Attempt 1/5 - Request timed out. Retrying...
Predicted PR Lifetime: 4
Correct PR type: 2
Attempt 1/5 - Request timed out. Retrying...
Attempt 2/5 - Request timed out. Retrying...
Predicted PR Lifetime: 1
Correct PR type: 2
Predicted PR Lifetime: 2
Correct PR type: 2
Predicted PR Lifetime: 2
Correct PR type: 1
Predicted PR Lifetime: 2
Correct PR type: 2
Attempt 1/5 - Request timed out. Retrying...
Attempt 2/5 - Request timed out. Retrying...
Predicted PR Lifetime: 2
Correct PR type: 3
A

In [21]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score


# Calculate weighted average F1-score, precision, and recall
f1 = f1_score(y_true, y_pred, labels=labels, average='weighted')
precision = precision_score(y_true, y_pred, labels=labels, average='weighted')
recall = recall_score(y_true, y_pred, labels=labels, average='weighted')

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=labels)

cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Calculate TP, FP, FN, TN
results = {}
for i, label in enumerate(labels):
    results[label] = {'TP': cm[i, i]}
    results[label]['FP'] = cm[:, i].sum() - cm[i, i]
    results[label]['FN'] = cm[i, :].sum() - cm[i, i]
    results[label]['TN'] = cm.sum() - (results[label]['TP'] + results[label]['FP'] + results[label]['FN'])

# Print results_fb
for label, metrics in results.items():
    print(f"{label}: {metrics}")

# Save results_fb to CSV
results_df = pd.DataFrame(results).T
results_df['F1-score'] = f1
results_df['Recall'] = recall
results_df['Precision'] = precision

#results_df.to_csv('confusion_matrix.csv')

print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"F1-score = {f1}")

#print accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy = {accuracy}")

1: {'TP': np.int64(384), 'FP': np.int64(65), 'FN': np.int64(63), 'TN': np.int64(1122)}
2: {'TP': np.int64(695), 'FP': np.int64(195), 'FN': np.int64(61), 'TN': np.int64(683)}
3: {'TP': np.int64(208), 'FP': np.int64(22), 'FN': np.int64(132), 'TN': np.int64(1272)}
4: {'TP': np.int64(65), 'FP': np.int64(0), 'FN': np.int64(26), 'TN': np.int64(1543)}
5: {'TP': np.int64(0), 'FP': np.int64(0), 'FN': np.int64(0), 'TN': np.int64(1634)}
Precision = 0.839123221374143
Recall = 0.8274173806609547
F1-score = 0.8234620284431489
Accuracy = 0.8274173806609547


/Users/atu/Library/Python/3.12/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/atu/Library/Python/3.12/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/atu/Library/Python/3.12/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
